In [144]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import classification_report
import eli5
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.metrics import accuracy_score

In [146]:
df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/bridges-to-prosperity-ds-a/britne/Data/finalclean.csv')

In [147]:
df

,Unnamed: 0,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,...,2013-14: Distance of bridge location from traditional crossing point (Meters),2013-14: Proposed bridge span (Meters),2013-14: Height differnce between banks (Meters),2013-14: Foundation space,2013-14: Space between the lowest point of bridge and highest flood level,2013-14: Confluence space available?,2013-14: Rejection Comments,Market access blocked by river,Education access blocked by river,Health access blocked by river
0,0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,Cancelled,-2.760833,29.488056,Suspension Bridge,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unavailable,0,0,0
1,1,Kagarama,1007325,NaN,Western Province,Nyamasheke,Complete,-2.322534,29.141944,Suspension Bridge,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unavailable,1,1,0
2,2,Karambi,1007374,NaN,Southern Province,Huye,Complete,-2.513333,29.612778,Suspension Bridge,...,10,70,0.06,sufficient,sufficient,no,Unavailable,1,1,1
3,3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.476407,29.420800,Suspension Bridge,...,1,55,0.2,sufficient,sufficient,no,Unavailable,1,1,1
4,4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.233022,29.575130,Suspension Bridge,...,3,30,0.02,sufficient,insufficient,yes,Unavailable,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,1467,Ngoma,1012951,Rwanda Needs Assessment 2018,Southern Province,Muhanga,Under Construction,-2.049360,29.707098,Suspension Bridge,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unavailable,1,1,1
1468,1468,Nyabikeri,1014031,Rwanda Needs Assessment 2018,Western Province,Karongi,Under Construction,-2.209275,29.471450,Other,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unavailable,1,1,1
1469,1469,Mugonero,1014056,Rwanda Needs Assessment 2018,Western Province,Karongi,Under Construction,-2.147630,29.344680,Suspension Bridge,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unavailable,1,1,1
1470,1470,Gasharu,1014069,Rwanda Needs Assessment 2018,Western Province,Karongi,Under Construction,-2.133330,29.362500,Suspension Bridge,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unavailable,1,1,1


In [148]:
df.shape

(1472, 68)

In [149]:
#this takes the target and transforms it into better data to predict

positive = (
    (df['Senior Engineering Review Conducted']=='Yes') & 
    (df['Bridge Opportunity: Stage'].isin(['Complete', 'Prospecting', 'Confirmed']))
)

negative = (
    (df['Senior Engineering Review Conducted']=='Yes') & 
    (df['Bridge Opportunity: Stage'].isin(['Rejected', 'Cancelled']))
)

unknown = df['Senior Engineering Review Conducted'].isna()

df.loc[positive, 'Good Site'] = 1
df.loc[negative, 'Good Site'] = 0

df.loc[unknown, 'Good Site'] = -1

df = df.drop(columns=['Senior Engineering Review Conducted', 'Bridge Opportunity: Stage'])

df['Good Site'].value_counts()

-1.0    1383
 1.0      65
 0.0      24
Name: Good Site, dtype: int64

In [166]:
#takes each row that contains -1(unknown) and creates a prediction then repalces the -1 with the prediction from the model
for index in df.index:
    if df.loc[index,'Good Site'] == -1:
        row = df.iloc[[index]]
        target = 'Good Site'
        train_features = train.drop(columns=[target])
        numeric_features = train_features.select_dtypes(include='number').columns.tolist()
        cardinality = train_features.select_dtypes(exclude='number').nunique()
        categorical_features = cardinality[cardinality <= 50].index.tolist()
        features = numeric_features + categorical_features

        row = row[features]
        test_pred = loaded_model.predict(row)

        new_new_pred = convert_pred(test_pred)
        df.loc[index, 'Good Site'] = new_new_pred

In [170]:
df['Good Site'].value_counts()

0.0    1170
1.0     302
Name: Good Site, dtype: int64

In [171]:
#created a new csv with the new predictions in the Good Site column
# df.to_csv('new_final.csv', index=False)

In [124]:
#creates a new dataframe that only contains the unknown or -1 rows
y_unknown = df[df['Good Site'] == -1]

#this gets rid of the unknown or -1 rows and only keeps the known positive or negative reviews
# df = df[df['Good Site'] > -1]

In [125]:
#separates the numeric with the other non-numeric columns with a cardinatily limiter

target = 'Good Site'
train_features = train.drop(columns=[target])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 50].index.tolist()
features = numeric_features + categorical_features

In [126]:
test1 = y_unknown[features]

In [127]:
test1.shape

(1383, 59)

In [128]:
# loaded_model = pickle.load(open('model3.pkl', 'rb'))
# test_pred = loaded_model.predict(test1)

In [164]:
#takes the predictions and converts it into solid 1 or 0
def convert_pred(array):
    new_list = []
    for i in array:
        if i >= 0.5:
            new_list.append(1)
        elif i < .5:
            new_list.append(0)
    return new_list

In [131]:
new_new_pred = convert_pred(test_pred)

In [132]:
new_df = pd.DataFrame(data=new_new_pred, columns=['Predictions'])

In [134]:
y_unknown['Good Site'] = new_df['Predictions'].values

In [48]:
df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/bridges-to-prosperity-ds-a/britne/Data/finalclean.csv')

positive = (
    (df['Senior Engineering Review Conducted']=='Yes') & 
    (df['Bridge Opportunity: Stage'].isin(['Complete', 'Prospecting', 'Confirmed']))
)

negative = (
    (df['Senior Engineering Review Conducted']=='Yes') & 
    (df['Bridge Opportunity: Stage'].isin(['Rejected', 'Cancelled']))
)

unknown = df['Senior Engineering Review Conducted'].isna()

df.loc[positive, 'Good Site'] = 1
df.loc[negative, 'Good Site'] = 0

df.loc[unknown, 'Good Site'] = -1

df = df.drop(columns=['Senior Engineering Review Conducted', 'Bridge Opportunity: Stage'])

df['Good Site'].value_counts()

-1.0    1383
 1.0      65
 0.0      24
Name: Good Site, dtype: int64

In [24]:
train, val = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df['Good Site'], random_state=42)

In [182]:
target = 'Good Site'
train_features = train.drop(columns=[target])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 50].index.tolist()
features = numeric_features + categorical_features

In [183]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [208]:
#this is the actual model for the project
model = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    SelectKBest(f_classif, k=15), 
    Ridge(alpha=1)
)

In [209]:
%%time
model.fit(X_train, y_train)

print('training accuracy:', model.score(X_train, y_train))
print('validation accuracy:', model.score(X_val, y_val))

training accuracy: 0.8404287074098808
validation accuracy: 0.7033026957939383
CPU times: user 447 ms, sys: 492 µs, total: 447 ms
Wall time: 440 ms


In [210]:
pickle.dump(model, open('model3.pkl', 'wb'))

In [211]:
loaded_model = pickle.load(open('model3.pkl', 'rb'))
loaded_model.predict(X_val)

array([ 0.99674953,  0.40973165,  0.18204627,  0.1576493 ,  0.97566146,
        0.93424708,  0.27584436,  0.93644547,  0.94304065,  1.08903043,
        1.21021593,  0.97867383,  0.18233999,  1.08243525, -0.09663145,
        1.12697284,  0.97940662,  0.94157506])